In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
import tensorflow as tf
import numpy as np
from  datetime import datetime

2025-01-13 23:26:25.361731: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 23:26:25.371360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736807185.381725  354528 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736807185.384817  354528 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 23:26:25.397042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model = tf.saved_model.load("../models/camera0/segmentation/saved_model")

I0000 00:00:1736807192.282409  354528 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8829 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
sig0 = model.signatures["serving_default"].structured_input_signature
display(sig0)
sig = (
    (),
    {
        "input_image": sig0[1]["input_image"],
        "input_image_meta": sig0[1]["input_image_meta"],
        "input_anchors": sig0[1]["input_anchors"],
    },
)
sig

((),
 {'input_anchors': TensorSpec(shape=(None, None, 4), dtype=tf.float32, name='input_anchors'),
  'input_image': TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_image'),
  'input_image_meta': TensorSpec(shape=(None, 14), dtype=tf.float32, name='input_image_meta')})

((),
 {'input_image': TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_image'),
  'input_image_meta': TensorSpec(shape=(None, 14), dtype=tf.float32, name='input_image_meta'),
  'input_anchors': TensorSpec(shape=(None, None, 4), dtype=tf.float32, name='input_anchors')})

In [4]:
from pathlib import Path

with Path("./input.py.bin").open() as f:
    c = f.read()
exec(c)

images = images.astype(np.float32)
image_metas = image_metas.astype(np.float32)
anchors = anchors.astype(np.float32)
display(anchors)
print("image_metas", image_metas)
display(detections.shape)
display(detections)

array([[[-0.02211869, -0.01105934,  0.02114117,  0.01008183],
        [-0.01564027, -0.01564027,  0.01466276,  0.01466276],
        [-0.01105934, -0.02211869,  0.01008183,  0.02114117],
        ...,
        [ 0.5845174 ,  0.7614669 ,  1.2913378 ,  1.1143883 ],
        [ 0.68817204,  0.68817204,  1.1876833 ,  1.1876833 ],
        [ 0.7614669 ,  0.5845174 ,  1.1143883 ,  1.2913378 ]]],
      dtype=float32)

image_metas [[0.000e+00 1.024e+03 1.024e+03 3.000e+00 1.024e+03 1.024e+03 3.000e+00
  0.000e+00 0.000e+00 1.024e+03 1.024e+03 1.000e+00 0.000e+00 0.000e+00]]


(1, 10, 6)

array([[[0.31554943, 0.60850537, 0.45372224, 0.69937855, 1.        ,
         0.99838877],
        [0.37167203, 0.4351973 , 0.39085433, 0.4648971 , 1.        ,
         0.8981155 ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ]]], dtype=float32)

In [73]:
input_image = tf.convert_to_tensor(images.astype(np.float32), name="input_image")
# input_image = tf.convert_to_tensor(np.random.uniform(0,1,size=(1,1024,1024,3)).astype(np.float32), name="input_image")

input_image_meta = tf.convert_to_tensor(
    image_metas.astype(np.float32), name="input_image_meta"
)
input_anchors = tf.convert_to_tensor(anchors.astype(np.float32), name="input_anchors")

result = model([input_image, input_image_meta, input_anchors])
display(result[0].shape)
display(result[1].shape)
display(result)

TensorShape([1, 10, 6])

TensorShape([1, 1000, 2])

[<tf.Tensor: shape=(1, 10, 6), dtype=float32, numpy=
 array([[[0.31555408, 0.60850465, 0.45372626, 0.6993831 , 1.        ,
          0.9983876 ],
         [0.371672  , 0.43519884, 0.39085475, 0.46489704, 1.        ,
          0.89809173],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ]]], dtype=float32)>

In [80]:
import onnx

onnx_model_path = "../models/camera0/segmentation/saved_model.onnx"
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

import onnxruntime as ort

ort_sess = ort.InferenceSession(onnx_model_path)
outputs = ort_sess.run(
    None,
    {"input_image": images, "input_image_meta": image_metas, "input_anchors": anchors},
)

# Print Result
for i, output in enumerate(outputs):
    print(f"Output #{i}")
    display(type(output))
    display(output.shape)
    display(output[0, 0])

2025-01-13 14:31:10.931094008 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
2025-01-13 14:31:10.931123914 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute
2025-01-13 14:31:10.931141336 [W:onnxruntime:Default, upsamplebase.h:178 UpsampleBase] `tf_half_pixel_for_nn` is deprecated since opset 13, yet this opset 13 model uses the deprecated attribute


Output #0


numpy.ndarray

(1, 1000, 4)

array([0.3164233 , 0.60936385, 0.45559508, 0.70325935], dtype=float32)

...

array([ 0.11690706, -0.2829972 , -1.7754984 , -1.5602344 ], dtype=float32)

Output #6


numpy.ndarray

(1, 261888, 2)

array([0.97783536, 0.02216464], dtype=float32)